# Prepare jobs for tool comparison of real data (IMG/VR4) 
We'll test the different aligners/search software on the real data from IMG/VR4.  
The original workflow used in IMG/VR4 is described in article [DOI: 10.1093/nar/gkac1037](https://doi.org/10.1093/nar/gkac1037) and is available in the [workflow](https://github.com/jgi-microbiome-data-science/crispr-host-prediction).  
That workflow uses a pre-filtered CRISPR sapcer set, the IMG/Vr4 virus sequence set, and spacer-protospacer pairs are identified via `blastn -w 8 --dust no` which is roughly equivalent to `blastn -task short` (what we will use here). Another KEY difference is the `--max_target_seqs` parameter, on the IMG/VR4 workflow is set to 1000, and here to some absurd highly (supposedly) unrealistic value (1000000).  
Note, we do not compare any filtereing done after the pairing was done (in the original workflow, a `assign_host.py` script preforms, among else, an evalution of the results). For that reason, we also do not benchmark hots assignment tools like spacerphaser, wish or iphop (even though they may be useful or utilize spacer-protospacer pairs).


To simplify the execution of the different tools, we'll use bash scripts generated by `bench.py` script, which we will modify to use the same contigs and spacers files, and discard the hyperfine. Instead, as the sequence set sizes are considerablt larger than the ones used by bnech.py simulations, we will run the different tools as seperate jobs via SLURM. Later (in [`Resource_usage_imgvr4.ipynb`](./Resource_usage_imgvr4.ipynb)), we will use get a general overview of the performance of the tools.

First we download the data

In [1]:
%%bash
mkdir imgvr4_data
cd imgvr4_data

Note, we'll use the IMG_VR_2022-09-20_7.1 version of IMG/VR (v4.1, with bugfix in UViG table and protein fasta), and specifically the high-confidence genomes only ( `IMGVR_all_nucleotides-high_confidence.fna`)


In [ ]:
%%bash
git clone https://github.com/jgi-microbiome-data-science/crispr-host-prediction 
cd crispr-host-prediction
# Download the database
mkdir crispr-db
cd crispr-db
wget https://portal.nersc.gov/cfs/m342/crisprDB/crispr_spacers_filtered_clustered.tsv --quiet

In [ ]:
%%bash
# Prepare the database files
cut -f1,14 crispr_spacers_filtered_clustered.tsv | sed 1d | sed 's/^/>/' | tr '\t' '\n' > spacers.fna
cut -f1,7 crispr_spacers_filtered_clustered.tsv > spacer_lineage.tsv
seqkit rmdup spacers.fna -s -i -j4 -o rm_duped_spacers.fna
seqkit seq -M100 imgvr4_data/crispr-host-prediction/crispr-db/rm_duped_spacers.fna -w0  > tmp
mv tmp imgvr4_data/crispr-host-prediction/crispr-db/rm_duped_spacers.fna
seqkit fx2tab -nl ./imgvr4_data/crispr-host-prediction/crispr-db/rm_duped_spacers.fna  > ./imgvr4_data/crispr-host-prediction/crispr-db/rm_duped_spacers_name_length.tab
# makeblastdb -in spacers.fna -out spacers.fna -dbtype nucl--quiet

In [ ]:
%%bash
cd ..
#login to https://genome.jgi.doe.gov/portal/pages/dynamicOrganismDownload.jsf?organism=IMG_VR
seqkit rmdup imgvr4_data/crispr-host-prediction/IMGVR_all_nucleotides-high_confidence.fna -s -i -j4 -o imgvr4_data/crispr-host-prediction/IMGVR_all_nucleotides-high_confidence_nr.fna
# wget https://portal.nersc.gov/genomad/__data__/IMGVR_DATA/imgvr4_nucleotide_db.tar.gz --quiet
wget https://portal.nersc.gov/genomad/__data__/IMGVR_DATA/IMGVR4_SEQUENCES.fna --quiet
# tar -xzf imgvr4_nucleotide_db.tar.gz    
# #BLAST viruses to database
#blastn -query test_viruses.fna -db crispr-db/spacers.fna -dust no -word_size 8 -max_target_seqs 1000 -outfmt '6 std qlen slen' -num_threads 64 > blastn.tsv

Next, we launch a dummy benchmark to generate the bash scripts, and make sure the tools installed and working.

In [12]:
%%bash
python bench.py -t 14 -nc 400 -ns 10 -lm 0 0 -ir 1 10 -prc 0.5 -mr 3

/clusterfs/jgi/groups/science/homes/uneri/.micromamba/envs/base_env/lib/python3.10/functools.py:889: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)



Running bowtie1...
Benchmark 1: bowtie-build results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_contigs.fa results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/bt1_contigs_index && bowtie --threads 14 -f --all -v 3 -x results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/bt1_contigs_index results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_spacers.fa -S results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//bowtie1_output.sam
  Time (mean ± σ):      6.175 s ±  0.224 s    [User: 4.420 s, System: 0.490 s]
  Range (min … max):    6.043 s …  6.434 s    3 runs
 



Successfully ran tool: bowtie1

Running bowtie2...
Benchmark 1: bowtie2-build --large-index --threads 14 results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_contigs.fa results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/contigs_bt2indx && bowtie2 --all --xeq --very-sensitive -x results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/contigs_bt2indx -f results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_spacers.fa -S results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//bowtie2_output.sam --threads 14
  Time (mean ± σ):      6.298 s ±  0.090 s    [User: 31.202 s, System: 2.964 s]
  Range (min … max):    6.207 s …  6.387 s    3 runs
 

Successfully ran tool: bowtie2

Running minimap2...
Benchmark 1: minimap2 -N 100 --eqx -t 14 -a results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_contigs.fa results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_sp


Successfully ran tool: blastn

Running mmseqs2...
Benchmark 1: mkdir -p results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_spacers results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_contigs results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_mmseqs results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_mmseqs_outputs && mmseqs createdb results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_spacers.fa results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_spacers/mmdb && mmseqs createdb results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/simulated_data/simulated_contigs.fa results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_contigs/mmdb && mmseqs search results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_spacers/mmdb results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//tmp_contigs/mmdb results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0

Parsing SAM file: 51it [00:00, 2973.24it/s]


...


Reading results for strobealign...
File size: 0.01 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//strobealign_output.sam exists
sam file looks right


Parsing SAM file: 59it [00:00, 119030.27it/s]



Reading results for blastn...
File size: 0.00 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//blastn_output.tsv exists

Reading results for mmseqs2...
File size: 0.00 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//mmseqs_output.tsv exists
qend < qstart detected, assuming tend and tstart are reversed

Reading results for spacer_containment...
File size: 0.00 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//spacer_containment_output.tsv exists

Reading results for mummer4...
File size: 0.01 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//mummer4_output.sam exists
@HD	VN:1.4	SO:unsorted
No reference file provided, using the smallest sam file present in the output file directory


Parsing SAM file: 51it [00:00, 108804.43it/s]



Reading results for lexicmap...
File size: 0.01 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//lexicmap_output.tsv exists

Reading results for vsearch...
File size: 0.00 MB
 output file  results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/raw_outputs//vsearch_output.blast6 exists
shape: (11, 7)
┌────────────────┬───────────────┬───────────────┬───────────────┬───────────┬──────────┬──────────┐
│ tool           ┆ true_positive ┆ false_positiv ┆ ground_truth_ ┆ precision ┆ recall   ┆ f1_score │
│ ---            ┆ s             ┆ es            ┆ total         ┆ ---       ┆ ---      ┆ ---      │
│ str            ┆ ---           ┆ ---           ┆ ---           ┆ f64       ┆ f64      ┆ f64      │
│                ┆ i64           ┆ i64           ┆ i64           ┆           ┆          ┆          │
╞════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════╪══════════╪══════════╡
│ bowtie1        ┆ 51            ┆ 0             ┆ 51       

Next, we'll modify the bash scripts to use the real data

In [ ]:
%%bash
cd /clusterfs/jgi/scratch/science/metagen/neri/code/blits/spacer_matching_bench
mkdir -p results/real_data/
mkdir -p results/real_data/slurm_logs/ results/real_data/results/ results/real_data/bash_scripts/ results/real_data/raw_outputs/ results/real_data/simulated_data/

for script in $(ls results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5/bash_scripts/*.sh); do
    new_script=results/real_data/bash_scripts/$(basename $script)
    cp $script results/real_data/bash_scripts/
    echo $new_script
    sed -i '1s|^|#!/bin/bash\n|' $new_script
    sed -i '2s|^|cd /clusterfs/jgi/scratch/science/metagen/neri/code/blits/spacer_bench/\n|' $new_script
    
    # Replace all variations of the run_t path
    sed -i 's|results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5|results/real_data|g' $new_script
    sed -i 's|./results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5|results/real_data|g' $new_script
    sed -i 's|/results/run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5|/results/real_data|g' $new_script
    sed -i 's|run_t_14_nc_400_ns_10_ir_1_10_lm_0_0_prc_0.5|real_data|g' $new_script
    
    # Replace data paths
    sed -i 's|results/real_data/simulated_data/simulated_contigs.fa|imgvr4_data/crispr-host-prediction/IMGVR_all_nucleotides-high_confidence_nr.fna|g' $new_script
    sed -i 's|results/real_data/simulated_data/simulated_spacers.fa|imgvr4_data/crispr-host-prediction/crispr-db/rm_duped_spacers.fna|g' $new_script
    
    # Remove hyperfine wrapper
    sed -i 's/hyperfine.*json '\''\(.*\)'\''/\1/' $new_script
    
    # Change thread count
    sed -i 's|14|"$1"|g' $new_script
    
    # Split && into multiple lines
    sed -i 's/ && /\n/g' $new_script
done

results/real_data/bash_scripts/bbmap_skimmer.sh


results/real_data/bash_scripts/blastn.sh
results/real_data/bash_scripts/bowtie1.sh
results/real_data/bash_scripts/bowtie2.sh
results/real_data/bash_scripts/lexicmap.sh
results/real_data/bash_scripts/minimap2.sh
results/real_data/bash_scripts/mmseqs.sh
results/real_data/bash_scripts/mummer4.sh
results/real_data/bash_scripts/spacer_containment.sh
results/real_data/bash_scripts/strobealign.sh
results/real_data/bash_scripts/vsearch.sh


Next we'll make a slurm scripts to submit the tools as seperate jobs.

In [3]:
%%bash
mkdir -p results/real_data/job_scripts/
for script in $(ls results/real_data/bash_scripts/*.sh); do
    tool_name=$(basename $script .sh)
    script_full_path=$(realpath $script)
    
    echo "#!/bin/bash
#SBATCH --mail-user=uneri@lbl.gov
#SBATCH --mail-type=FAIL,END,BEGIN
#SBATCH -A grp-org-sc-metagen
#SBATCH -q jgi_normal
#SBATCH -c 
#SBATCH --mem=168G 
## specify runtime
#SBATCH -t 72:00:00
## specify job name
#SBATCH -J "$tool_name"   
## specify output and error file
#SBATCH -o /clusterfs/jgi/scratch/science/metagen/neri/code/blits/spacer_bench/results/real_data/slurm_logs/"$tool_name"-%A.out
#SBATCH -e /clusterfs/jgi/scratch/science/metagen/neri/code/blits/spacer_bench/results/real_data/slurm_logs/"$tool_name"-%A.err
bash $script_full_path "\$"SLURM_CPUS_PER_TASK
" > results/real_data/job_scripts/"$tool_name".sh
done

and run the tools as seperate jobs.

In [4]:
%%bash
for script in $(ls results/real_data/job_scripts/*.sh); do
    echo $script
    sbatch $script
done

results/real_data/job_scripts/bbmap_skimmer.sh
results/real_data/job_scripts/blastn.sh
results/real_data/job_scripts/bowtie1.sh
results/real_data/job_scripts/bowtie2.sh
results/real_data/job_scripts/lexicmap.sh
results/real_data/job_scripts/minimap2.sh
results/real_data/job_scripts/mmseqs.sh
results/real_data/job_scripts/mummer4.sh
results/real_data/job_scripts/spacer_containment.sh
results/real_data/job_scripts/strobealign.sh
results/real_data/job_scripts/vsearch.sh


Next (in [`Performance_imgvr4.ipynb`](./Performance_imgvr4.ipynb)), we'll use functions from bench.py to read the mapping/search results and compare them.